In [1]:
%reset -fs

In [2]:
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import *
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
pd.set_option('display.max_columns', None)

In [3]:
spark = SparkSession.builder.appName('GCSFilesRead').config("spark.driver.memory", "16g").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/04 17:23:05 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark._jsc.hadoopConfiguration().set("google.cloud.auth.service.account.json.keyfile","root-quasar-182016-a55c6c32176d.json")

In [5]:
bucket_name="ndir-metis-bucket"
path=f"gs://{bucket_name}/asteroid/Asteroid_Updated.csv"

In [6]:
df=spark.read.csv(path, sep=',', inferSchema=True, header=True)
df.printSchema()

root
 |-- name: string (nullable = true)
 |-- a: double (nullable = true)
 |-- e: double (nullable = true)
 |-- i: double (nullable = true)
 |-- om: double (nullable = true)
 |-- w: double (nullable = true)
 |-- q: double (nullable = true)
 |-- ad: double (nullable = true)
 |-- per_y: double (nullable = true)
 |-- data_arc: integer (nullable = true)
 |-- condition_code: string (nullable = true)
 |-- n_obs_used: integer (nullable = true)
 |-- H: double (nullable = true)
 |-- neo: string (nullable = true)
 |-- pha: string (nullable = true)
 |-- diameter: string (nullable = true)
 |-- extent: string (nullable = true)
 |-- albedo: double (nullable = true)
 |-- rot_per: double (nullable = true)
 |-- GM: double (nullable = true)
 |-- BV: double (nullable = true)
 |-- UB: double (nullable = true)
 |-- IR: double (nullable = true)
 |-- spec_B: string (nullable = true)
 |-- spec_T: string (nullable = true)
 |-- G: double (nullable = true)
 |-- moid: double (nullable = true)
 |-- class: string (

# Feature Info
- a: semi-major axis (au)
- e: eccentricity
- i: inclination with respect to ecliptic plane
- om: longitude of the ascending node
- w: argument of perihelion
- q: perihelion distance (au)
- ad: aphelion distance (au)
- per_y: orbital period (years)
- data_arc: span of recorded data (days)
- condition_code: orbital condition code
- n_obs_used: number of observations used
- H: absolute magnitude parameter
- neo: near-earth object
- pha: physically hazardous object
- diameter: diameter (target variable)
- extent: Object bi/tri axial ellipsoid dimensions(Km)
- albedo: geometric albedo
- rot_per: rotation period (hours)
- GM: gravitational parameter. Product of mass and gravitational constant
- BV: Color index B-V magnitude difference
- UB: Color index U-B magnitude difference
- IR: Color index I-R magnitude difference
- specB: Spectral taxonomic type(SMASSII)
- specT: Spectral taxonomic type (Tholen)
- G: Magnitude slope parameter
- moid: Earth minimum orbit intersection distance
- class: asteroid orbit class
- n: mean motion (degrees/day)
- per: orbital period (days)
- ma: mean ananomly (degrees)


In [7]:
def preview(df, n=20):
    return pd.DataFrame(df.take(n), columns=df.columns)
preview(df)

22/04/04 17:23:58 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


,name,a,e,i,om,w,q,ad,per_y,data_arc,condition_code,n_obs_used,H,neo,pha,diameter,extent,albedo,rot_per,GM,BV,UB,IR,spec_B,spec_T,G,moid,class,n,per,ma
0,Ceres,2.769165,0.076009,10.594067,80.305532,73.597694,2.558684,2.979647,4.608202,8822,0,1002,3.34,N,N,939.4,964.4 x 964.2 x 891.8,0.0900,9.074170,62.6284,0.713,0.426,None,C,G,0.12,1.594780,MBA,0.213885,1683.145708,77.372096
1,Pallas,2.772466,0.230337,34.836234,173.080063,310.048857,2.133865,3.411067,4.616444,72318,0,8490,4.13,N,N,545,582x556x500,0.1010,7.813200,14.3000,0.635,0.284,None,B,B,0.11,1.233240,MBA,0.213503,1686.155999,59.699133
2,Juno,2.669150,0.256942,12.988919,169.852760,248.138626,1.983332,3.354967,4.360814,72684,0,7104,5.33,N,N,246.596,None,0.2140,7.210000,NaN,0.824,0.433,None,Sk,S,0.32,1.034540,MBA,0.226019,1592.787285,34.925016
3,Vesta,2.361418,0.088721,7.141771,103.810804,150.728541,2.151909,2.570926,3.628837,24288,0,9325,3.20,N,N,525.4,572.6 x 557.2 x 446.4,0.4228,5.342128,17.8000,0.782,0.492,None,V,V,0.32,1.139480,MBA,0.271609,1325.432765,95.861936
4,Astraea,2.574249,0.191095,5.366988,141.576605,358.687607,2.082324,3.066174,4.130323,63507,0,2916,6.85,N,N,106.699,None,0.2740,16.806000,NaN,0.826,0.411,None,S,S,NaN,1.095890,MBA,0.238632,1508.600458,282.366289
5,Hebe,2.425160,0.203007,14.737901,138.640203,239.807490,1.932835,2.917485,3.776755,62329,0,6034,5.71,N,N,185.18,None,0.2679,7.274500,NaN,0.822,0.399,None,S,S,0.24,0.973965,MBA,0.260972,1379.459705,86.197923
6,Iris,2.385334,0.231206,5.523651,259.563231,145.265106,1.833831,2.936837,3.684105,62452,0,5206,5.51,N,N,199.83,None,0.2766,7.139000,NaN,0.855,0.484,None,S,S,NaN,0.846100,MBA,0.267535,1345.619196,140.419656
7,Flora,2.201764,0.156499,5.886955,110.889330,285.287462,1.857190,2.546339,3.267115,62655,0,2744,6.49,N,N,147.491,None,0.2260,12.865000,NaN,0.885,0.489,None,None,S,0.28,0.874176,MBA,0.301681,1193.313717,194.882895
8,Metis,2.385637,0.123114,5.576816,68.908577,6.417369,2.091931,2.679342,3.684806,61821,0,2649,6.28,N,N,190,None,0.1180,5.079000,NaN,0.858,0.496,None,None,S,0.17,1.106910,MBA,0.267484,1345.875362,276.861623
9,Hygiea,3.141539,0.112461,3.831560,283.202167,312.315206,2.788240,3.494839,5.568291,62175,0,3409,5.43,N,N,407.12,None,0.0717,27.630000,7.0000,0.696,0.351,None,C,C,NaN,1.778390,MBA,0.177007,2033.818284,152.184851


In [9]:
astro_df = df.dropna(subset=['diameter'])

In [11]:
astro_df = astro_df.drop('extent', 'rot_per', 'GM', 'BV', 'UB', 'IR', 'spec_B', 'spec_T', 'G')

In [12]:
astro_df = astro_df.drop('name', 'data_arc', 'H', 'albedo')

In [13]:
def trim(string):
    return (string
    .replace('Y', 'True')
    .replace('N', 'False')
    )


In [14]:
trim = udf(trim)


In [15]:
for dtype in astro_df.dtypes:
    column = dtype[0]
    if dtype[1] == 'string':
        astro_df = astro_df.withColumn(column, trim(astro_df[column]))

In [16]:
astro_df = astro_df.withColumn('diameter', astro_df['diameter'].cast('integer').cast('double'))

In [17]:
astro_df = astro_df.withColumn('n_obs_used', astro_df['n_obs_used'].cast('double'))

In [18]:
for column in ['neo', 'pha']:
    astro_df = astro_df.withColumn(column, astro_df[column].cast('boolean').cast('int'))

In [19]:
condition_code_indexer = StringIndexer(inputCol="condition_code", outputCol="condition_codeIndex")
class_indexer = StringIndexer(inputCol="class", outputCol="classIndex")
onehotencoder_condition_code_vector = OneHotEncoder(inputCol="condition_codeIndex", outputCol="condition_code_vec")
onehotencoder_class_vector = OneHotEncoder(inputCol="classIndex", outputCol="class_vec")

#Create pipeline and pass all stages
encoding_pipeline = Pipeline(stages=[condition_code_indexer,
                            class_indexer,
                            onehotencoder_condition_code_vector,
                            onehotencoder_class_vector
                    ])

In [20]:
astro_df = encoding_pipeline.fit(astro_df).transform(astro_df)

In [21]:
astro_df = astro_df.drop('condition_code', 'class', 'condition_codeIndex', 'classIndex')

In [22]:
astro_df = astro_df.select("a","e","i",'om','w','q','ad', 'per_y', 'n_obs_used', 'neo', 'pha', 'moid', 'n', 'per', 'ma', 'condition_code_vec', 'class_vec', 'diameter')

In [23]:
train_df, val_df, test_df = astro_df.randomSplit([0.6, 0.2, 0.2], seed=42)

In [24]:
features = astro_df.schema.names[:-1]
features

['a',
 'e',
 'i',
 'om',
 'w',
 'q',
 'ad',
 'per_y',
 'n_obs_used',
 'neo',
 'pha',
 'moid',
 'n',
 'per',
 'ma',
 'condition_code_vec',
 'class_vec']

In [25]:
assembler = VectorAssembler(inputCols=features, outputCol='features')

In [26]:
train_pack = assembler.transform(train_df)
val_pack = assembler.transform(val_df)
test_pack = assembler.transform(test_df)

In [27]:
for field in features:
    train_pack = train_pack.drop(field)
    val_pack = val_pack.drop(field)
    test_pack  = test_pack.drop(field)

In [28]:
val_pack.printSchema()

root
 |-- diameter: double (nullable = true)
 |-- features: vector (nullable = true)



In [30]:
gbt = GBTRegressor(featuresCol='features', labelCol='diameter', maxIter=10)

In [31]:
gbt_model = gbt.fit(train_pack)
val_pred = gbt_model.transform(val_pack)

In [32]:
val_pred = val_pred.select('prediction', 'diameter')

In [33]:
val_pred.printSchema()

root
 |-- prediction: double (nullable = false)
 |-- diameter: double (nullable = true)



In [34]:
evaluator = RegressionEvaluator(
    labelCol="diameter", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(val_pred)
rmse

22/04/04 17:27:20 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/04/04 17:27:20 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/04/04 17:27:24 ERROR Executor: Exception in task 0.0 in stage 113.0 (TID 853)
scala.MatchError: [29.342053008836395,null,1.0] (of class org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema)
	at org.apache.spark.ml.evaluation.RegressionEvaluator.$anonfun$getMetrics$1(RegressionEvaluator.scala:127)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.TraversableOnce.foldLeft(TraversableOnce.scala:199)
	at scala.collection.TraversableOnce.foldLeft$(Traversable

Py4JJavaError: An error occurred while calling o572.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 113.0 failed 1 times, most recent failure: Lost task 0.0 in stage 113.0 (TID 853) (nathaniels-mbp.lan executor driver): scala.MatchError: [29.342053008836395,null,1.0] (of class org.apache.spark.sql.catalyst.expressions.GenericRowWithSchema)
	at org.apache.spark.ml.evaluation.RegressionEvaluator.$anonfun$getMetrics$1(RegressionEvaluator.scala:127)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator$$anon$10.next(Iterator.scala:461)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at scala.collection.TraversableOnce.foldLeft(TraversableOnce.scala:199)
	at scala.collection.TraversableOnce.foldLeft$(TraversableOnce.scala:192)
	at scala.collection.AbstractIterator.foldLe